In [1]:
import sys
sys.path.insert(0, "../..")

from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.convergence import convergence_in_time, convergence_in_space
from MPyDATA_examples.Olesik_et_al_2020.distributions import n_n, n_s
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2
from MPyDATA_examples.utils.error_norms import L2
from MPyDATA_examples.utils.show_plot import show_plot

import matplotlib.pyplot as plt
import matplotlib

import numpy as np

In [2]:
opt_sets = [({'n_iters': 1},), 
            ({'n_iters': 1},{'n_iters':2},{'n_iters':3}),
            ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True}),
            ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True, 'fct': True}),
            ({'n_iters': 1},{'n_iters':2},{'n_iters':2, 'iga':True, 'fct': True, 'tot': True})
           ]

In [3]:
def compute_and_plot(psi_coord, grid_layout, n_bins, dt, opt_set, plots):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        dt=dt,
        opt_set=opt_set,
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 18})
    for coord in results.keys():
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()): 
            plot_data = results[coord]['numerical'][opts]
            for i in range(len(setup.nt)):
                mnorm = setup.mixing_ratios[i]
                t = setup.nt[i] * setup.dt
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth =.7
                    )
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    mnorm=mnorm, color = colors[opt_i]
                )
        print(f"grid_layout: {coord}")
        show_plot()

In [5]:
compute_and_plot(psi_coord=x_p2(), grid_layout=x_p2(), dt=1, n_bins=64, opt_set=({"n_iters":1},), plots=['m'])

../../MPyDATA_examples/utils/error_norms.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  sum(pow(numerical - analytical, 2)) / nt / nx


grid_layout: x_p2
